In [1]:
#!/usr/bin/python2.6  
# -*- coding: utf-8 -*-  
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time
import math

plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签


from tensorflow import keras
from tensorflow.keras.preprocessing import image
print(tf.__version__)

2.0.0


In [2]:
train_dir = "/opt/tv/training/training"
valid_dir = "/opt/tv/validation/validation"

dir1 = os.listdir(train_dir)

dir2 = os.listdir(valid_dir)

height = 128
width = 128
channels = 3
batch_size = 32
num_classes = 20

In [3]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    shear_range = 0.2,
    zoom_range = 0.2,
    channel_shift_range=10,
    brightness_range=[0.1, 1],
    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = "categorical")

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = False,
                                                    class_mode = "categorical")

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num)
print(valid_num)

Found 893 images belonging to 20 classes.
Found 56 images belonging to 20 classes.
893
56


In [ ]:
model = keras.models.Sequential([

    keras.layers.Conv2D(filters=16, kernel_size=3, padding='same',
                        activation='relu', input_shape=[width, height, channels]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=16, kernel_size=3, padding='same',
                        activation='relu'),
    #keras.layers.SpatialDropout2D(0.5),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu'),
    #keras.layers.SpatialDropout2D(0.5),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    #keras.layers.SpatialDropout2D(0.5),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    #keras.layers.SpatialDropout2D(0.5),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 16)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        9

In [ ]:
epochs = 40
history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num // batch_size)

Epoch 1/40
27/27 [==============================] - 16s 584ms/step - loss: 4.0788 - accuracy: 0.2091 - val_loss: 3.0771 - val_accuracy: 0.0000e+00
Epoch 2/40
13/27 [=============>................] - ETA: 7s - loss: 2.2793 - accuracy: 0.4062

In [ ]:
def plot_learning_curves(history, label, epcohs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 9)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
    directory = '/opt/tv/validation/validation',
    classes = dir1,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = "categorical")

In [ ]:
test_predict = model.predict_generator(test_generator,
                                       workers = 10,
                                       use_multiprocessing = True)

In [ ]:
filenames = test_generator.filenames
fullfilenames = test_generator.filepaths
pre_class = test_generator.labels

train_labels = train_generator.class_indices
labels1 = dict(zip(train_labels.values(),train_labels.keys()))

test_labels = test_generator.class_indices
labels2 = dict(zip(test_labels.values(),test_labels.keys()))
error_paths = []
error_labels = []
for i in range(len(test_predict)):
    pre = np.argmax(test_predict[i])
    if(labels2[pre_class[i]] != labels1[pre]):
        error_paths.append(fullfilenames[i])
        error_labels.append(labels1[pre])
        print(fullfilenames[i] + ' -> ' + labels1[pre])

In [ ]:
model.evaluate_generator(valid_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [ ]:
'''
    传入一个目录名列表，一个标签列表，进行显示
'''

def show_imgs_labels(n_cols, error_paths, error_labels):
    assert len(error_paths) == len(error_labels)
    max_num = len(error_paths)
    n_rows = math.ceil(max_num / n_cols)    #向上取整
    plt.figure(figsize = (n_cols * 1.4, n_rows * 1.6))
    
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols * row + col 
            if index >= max_num:    #如果画完了就退出
                break
            temp_img = image.load_img(error_paths[index], target_size = (128, 128))
            plt.subplot(n_rows, n_cols, index+1)
            plt.imshow(temp_img, cmap="binary",
                       interpolation = 'nearest')
            plt.axis('off')
            plt.title(error_labels[index])
    plt.show()
    
show_imgs_labels(10, error_paths, error_labels)